In [1]:
import cv2
import numpy as np
import mediapipe as mp
from playsound import playsound
import time
import threading
import pygame
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.9.21)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
pygame.init()
pygame.mixer.init()

In [7]:
import cv2
import pygame
import threading
import time
import mediapipe as mp
import numpy as np
import random


# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose()
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils


# Initialize Pygame for sound
pygame.init()
pygame.mixer.init()
time.sleep(1) 

# Frame skipping
frame_skip = 5  # Process every 2nd frame
frame_count = 0
last_message_time = 0
message_to_display = ""
delay_time = 2 
# Global variables for sound
sound_playing = False
stop_sound = False
sound_thread = None

friendly_pose_time = None
alert_time = None

last_detection_time = 0

def play_alert_sound():
    """Plays the alert sound in a loop until stopped."""
    global stop_sound, sound_playing
    if not pygame.mixer.get_init():
        pygame.mixer.init()
        time.sleep(1)

    pygame.mixer.music.load("E:/pose detection model/siren-alert-96052.mp3")
    pygame.mixer.music.set_volume(1.0)
    pygame.mixer.music.play(-1)  # Loop indefinitely

    sound_playing = True  # Mark that sound is playing

    while not stop_sound and pygame.mixer.music.get_busy():
        time.sleep(0.1)

    pygame.mixer.music.stop()
    stop_sound = False
    sound_playing = False 


def start_sound():
    """Starts the alert sound in a new thread if not already playing."""
    global sound_thread, sound_playing
    if not sound_playing:  # Only start if the sound is not already playing
        sound_thread = threading.Thread(target=play_alert_sound, daemon=True)
        sound_thread.start()


def stop_sound_playing():
    """Stops the alert sound."""
    global stop_sound
    stop_sound = True




def is_person_backside(left_shoulder, right_shoulder, left_hip, right_hip, nose):


    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    
    # Check if nose is sufficiently visible
    nose_not_visible = nose.visibility < 0.7  # threshold to ensure nose is visible or not
    if nose_not_visible:
        return False  # If nose is not visible, it could be a top backside or another pose

    # Checking the relative position of the nose compared to shoulders and hips
    nose_x = nose.x
    left_shoulder_x = left_shoulder.x
    right_shoulder_x = right_shoulder.x
    left_hip_x = left_hip.x
    right_hip_x = right_hip.x
    
    # In a backside pose, the nose should be behind the shoulders and hips on the X-axis.
    # Shoulders and hips should be more in front or rotated.
    if nose_x < min(left_shoulder_x, right_shoulder_x) and nose_x < min(left_hip_x, right_hip_x):
        shoulder_x_diff = abs(left_shoulder_x - right_shoulder_x)
        # Ensure the shoulders are sufficiently rotated (relative to the hips)
        if shoulder_x_diff > 0.1:  # Arbitrary threshold for shoulder rotation
            return True  # Detected normal backside pose
    
    return False 




def is_backside_salute_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, head_top,tolerance=0.05):
    
  
    left_elbow, right_elbow = right_elbow, left_elbow
    left_wrist, right_wrist = right_wrist, left_wrist
    
    left_salute = (
        left_wrist.y < left_shoulder.y + tolerance and
        left_elbow.y > left_wrist.y - tolerance and
        left_elbow.y < left_shoulder.y + tolerance and
        left_wrist.y >= head_top.y - tolerance 
    )
    right_salute = (
        right_wrist.y < right_shoulder.y + tolerance and
        right_elbow.y > right_wrist.y - tolerance and
        right_elbow.y < right_shoulder.y + tolerance and
        right_wrist.y >= head_top.y - tolerance 
    )
    return left_salute or right_salute 




def is_backside_hands_up_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist):

        left_hands_up = (
        left_wrist.y < left_shoulder.y and  # ✅ Corrected condition
        left_elbow.y > left_wrist.y)

        right_hands_up = (
        right_wrist.y < right_shoulder.y and  # ✅ Corrected condition
        right_elbow.y > right_wrist.y)

        return left_hands_up and right_hands_up




def is_backside_jesus_christ_pose(left_shoulder, right_shoulder, left_wrist, right_wrist, tolerance=0.05):
    
    
    # Swap left and right for backside pose
    left_wrist, right_wrist = right_wrist, left_wrist
    left_shoulder, right_shoulder = right_shoulder, left_shoulder

    # Ensure wrists are not above head or below waist, and arms are expanded
    left_wrist_correct_height = left_wrist.y >= left_shoulder.y - tolerance
    right_wrist_correct_height = right_wrist.y >= right_shoulder.y - tolerance

    # Allow flexibility in arm extension (not strict)
    left_arm_extended = abs(left_wrist.x - left_shoulder.x) > 0.1  # Less strict
    right_arm_extended = abs(right_wrist.x - right_shoulder.x) > 0.1  # Less strict

    # Check if wrists are in correct positions (not above head, not below waist)
    wrists_in_correct_position = left_wrist.y >= left_shoulder.y and right_wrist.y >= right_shoulder.y

    return left_wrist_correct_height and right_wrist_correct_height and left_arm_extended and right_arm_extended and wrists_in_correct_position



def is_top_backside(landmarks):
    """
    Detect if the person is viewed from the top backside using the nose visibility and z-depth.
    """
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Nose visibility threshold
    nose_not_visible = nose.visibility < 0.7

    # Shoulders and hips are closer to the camera than the nose
    shoulders_z = (left_shoulder.z + right_shoulder.z) / 2
    hips_z = (left_hip.z + right_hip.z) / 2
    head_closer_to_camera = shoulders_z < nose.z and hips_z < nose.z

    return nose_not_visible and head_closer_to_camera

def is_palm_open(hand_landmarks, hand_label):
    """
    Detect if the palm is open based on finger extension.
    Adjust thumb logic based on hand (left/right).
    """
    landmarks = hand_landmarks.landmark

    # Extract landmark positions
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
    thumb_mcp = landmarks[mp_hands.HandLandmark.THUMB_MCP]

    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_pip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_PIP]

    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_pip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]

    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP]
    ring_pip = landmarks[mp_hands.HandLandmark.RING_FINGER_PIP]

    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP]
    pinky_pip = landmarks[mp_hands.HandLandmark.PINKY_PIP]

    # Fix for left hand (MediaPipe flips x-coordinates)
    if hand_label == "Left":
        is_thumb_extended = thumb_tip.x < thumb_mcp.x  # Thumb moves left for open palm
    else:
        is_thumb_extended = thumb_tip.x > thumb_mcp.x  # Thumb moves right for open palm

    # Check if fingers are extended
    is_index_extended = index_tip.y < index_pip.y  
    is_middle_extended = middle_tip.y < middle_pip.y
    is_ring_extended = ring_tip.y < ring_pip.y
    is_pinky_extended = pinky_tip.y < pinky_pip.y

    # If all fingers are extended, return True (Open Palm)
    return is_thumb_extended and is_index_extended and is_middle_extended and is_ring_extended and is_pinky_extended
      
def is_top_backside_hands_up(landmarks):
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    left_hand_up = (
        left_wrist.z < left_shoulder.z - 0.1 and  # Closer to camera
        left_wrist.y < left_shoulder.y           # Above shoulder vertically
    )
    right_hand_up = (
        right_wrist.z < right_shoulder.z - 0.1 and
        right_wrist.y < right_shoulder.y
    )

    return left_hand_up and right_hand_up


def is_top_backside_jesus_christ(landmarks, shoulder_distance):
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    horizontal_condition = abs(left_wrist.x - right_wrist.x) > 1.5 * shoulder_distance
    vertical_alignment = abs(left_wrist.y - left_shoulder.y) < 0.2 * shoulder_distance
    z_depth_alignment = abs(left_wrist.z - right_wrist.z) < 0.1

    return horizontal_condition and vertical_alignment and z_depth_alignment

def is_top_backside_salute(landmarks):
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]

    left_salute = (
        left_wrist.z < nose.z and                    # Closer to camera than head
        abs(left_wrist.x - left_shoulder.x) < 0.3 and  # Near the shoulder horizontally
        left_wrist.y < left_shoulder.y               # Above the shoulder vertically
    )
    right_salute = (
        right_wrist.z < nose.z and
        abs(right_wrist.x - right_shoulder.x) < 0.3 and
        right_wrist.y < right_shoulder.y
    )

    return left_salute or right_salute



def is_salute_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, head_top,left_hand_open,right_hand_open,tolerance=0.05):
    """
    Detect salute pose.
    """
    left_salute = (
        left_wrist.y < left_shoulder.y + tolerance and
        left_elbow.y > left_wrist.y - tolerance and
        left_elbow.y < left_shoulder.y + tolerance and
        left_wrist.y >= head_top.y - tolerance and left_hand_open
    )
    right_salute = (
        right_wrist.y < right_shoulder.y + tolerance and
        right_elbow.y > right_wrist.y - tolerance and
        right_elbow.y < right_shoulder.y + tolerance and
        right_wrist.y >= head_top.y - tolerance  and right_hand_open
    )
    return left_salute or right_salute 




def is_hands_up_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist , left_hand_open,right_hand_open):

    

    left_hands_up = (
    left_wrist.y < left_shoulder.y and  # ✅ Corrected condition
    left_elbow.y > left_wrist.y)

    right_hands_up = (
    right_wrist.y < right_shoulder.y and  # ✅ Corrected condition
    right_elbow.y > right_wrist.y)

    return left_hands_up and right_hands_up and left_hand_open and right_hand_open




def is_jesus_christ_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, tolerance=0.05):
    """
    Detect Jesus Christ pose with improved accuracy.
    - Ensures wrists do not go above the shoulders.
    - Allows flexibility in arm extension.
    """

    # Ensure wrists are at or slightly below shoulder level
    left_wrist_correct_height = left_wrist.y >= left_shoulder.y - tolerance
    right_wrist_correct_height = right_wrist.y >= right_shoulder.y - tolerance

    # Allow flexibility in arm extension (not strict)
    left_arm_extended = abs(left_wrist.x - left_shoulder.x) > 0.1  # Less strict
    right_arm_extended = abs(right_wrist.x - right_shoulder.x) > 0.1  # Less strict

    return left_wrist_correct_height and right_wrist_correct_height and left_arm_extended and right_arm_extended



def is_top_down_hands_up(landmarks, shoulder_distance,left_hand_open,right_hand_open):
    """
    Detect hands-up pose in top-down view with stricter conditions.
    """
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    avg_shoulder_z = (left_shoulder.z + right_shoulder.z) / 2

    left_hand_up = (
        left_wrist.z < avg_shoulder_z - 0.2 * shoulder_distance and  # Stricter z-depth
        left_wrist.y < left_shoulder.y and  # Wrist higher than shoulder vertically
        abs(left_wrist.x - left_shoulder.x) < 0.5 * shoulder_distance  # Wrist near shoulder horizontally
    )
    right_hand_up = (
        right_wrist.z < avg_shoulder_z - 0.2 * shoulder_distance and
        right_wrist.y < right_shoulder.y and
        abs(right_wrist.x - right_shoulder.x) < 0.5 * shoulder_distance
    )
     
    return left_hand_up and right_hand_up and left_hand_open and right_hand_open




def is_top_down_t_pose(landmarks, shoulder_distance):
    """
    Detect T-pose in top-down view with stricter conditions.
    """
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    # Horizontal distance between wrists
    wrist_distance = np.abs(left_wrist.x - right_wrist.x)
    required_distance = 1.5 * shoulder_distance

    # Ensure wrists are horizontally far apart (key T-pose feature)
    horizontal_condition = wrist_distance > required_distance

    # Ensure wrists are at a similar vertical level (y-coordinates close)
    vertical_alignment = np.abs(left_wrist.y - right_wrist.y) < 0.1 * shoulder_distance

    # Ensure wrists are not too far above or below the shoulders
    left_wrist_level = np.abs(left_wrist.y - left_shoulder.y) < 0.2 * shoulder_distance
    right_wrist_level = np.abs(right_wrist.y - right_shoulder.y) < 0.2 * shoulder_distance

    # Ensure wrists have similar z-depth (not closer/farther from the camera)
    z_depth_alignment = np.abs(left_wrist.z - right_wrist.z) < 0.1

    # Combine all conditions
    return horizontal_condition and vertical_alignment and left_wrist_level and right_wrist_level and z_depth_alignment



# Initialize video capture
cap = cv2.VideoCapture(0)
time.sleep(3)  # Allow camera to warm up

cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Video", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Main loop


left_hand_open= False
right_hand_open = False
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    delay_time = 2  # Default delay for friend
    if frame_count % frame_skip == 0:
        results = pose.process(rgb_frame)
        results1 = hands.process(rgb_frame)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        if(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value] and
           landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]) or \
          (landmarks[mp_pose.PoseLandmark.NOSE.value]):
           start_sound()
        
        last_detection_time = time.time()
        landmarks = results.pose_landmarks.landmark
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        head_top = landmarks[mp_pose.PoseLandmark.NOSE.value]
        shoulder_distance = np.abs(left_shoulder.x - right_shoulder.x)
        waist_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].y
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
        


        is_backside = is_person_backside(left_shoulder, right_shoulder, left_hip, right_hip, nose)
    
        left_hand_landmarks = False
        right_hand_landmarks = False
    else:
        stop_sound_playing()  
        

    if results1.multi_hand_landmarks and results1.multi_handedness:
        for hand_idx, (hand_landmarks, handedness) in enumerate(zip(results1.multi_hand_landmarks, results1.multi_handedness)):
             wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            
            # Get MediaPipe's handedness
             mediapipe_label = handedness.classification[0].label  # "Left" or "Right"
            
            # Correct if needed (since webcam flips view)
             corrected_label = "Left" if mediapipe_label == "Right" else "Right"

            # Draw hand landmarks
             mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Check if palm is open or closed
             palm_open = is_palm_open(hand_landmarks, corrected_label)

            # Track which hands are open
             if corrected_label == "Left":
                left_hand_open = palm_open
             elif corrected_label == "Right":
                right_hand_open = palm_open

        label = ""  # Default initialization before conditions
        color = (255, 255, 255)
        
        if is_backside_salute_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, head_top,):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        elif is_backside_hands_up_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        elif is_backside_jesus_christ_pose(left_shoulder, right_shoulder,left_wrist, right_wrist):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        else:
                label, color = "Alert! Shooting Triggered!", (0, 0, 255)
                start_sound()

        

        if is_top_backside(landmarks):
          if is_top_backside_hands_up(landmarks):
             cv2.putText(frame, "Friendly Pose Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
             stop_sound_playing()
          elif is_top_backside_jesus_christ(landmarks, shoulder_distance):
             cv2.putText(frame, "Friendly Pose Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
             stop_sound_playing()
          elif is_top_backside_salute(landmarks):
             cv2.putText(frame, "Friendly Pose Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
             stop_sound_playing()
          else:
            cv2.putText(frame, "Alert! Shooting Triggered", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            start_sound()


        elif is_top_down_hands_up(landmarks, shoulder_distance,left_hand_open,right_hand_open):
            label, color = "Friendly Pose Detected", (0, 255, 0)
            stop_sound_playing()
        elif is_top_down_t_pose(landmarks, shoulder_distance):
            label, color = "Friendly Pose Detected", (0, 255, 0)
            stop_sound_playing()
        elif is_salute_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, head_top,left_hand_open,right_hand_open, tolerance=0.05):
               label, color = "Friendly Pose Detected", (0, 255, 0)
               stop_sound_playing()
        elif  is_hands_up_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist , left_hand_open, right_hand_open):
               label, color = "Friendly Pose Detected", (0, 255, 0)
               stop_sound_playing()
        elif is_jesus_christ_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist):
               label, color = "Friendly Pose Detected", (0, 255, 0)
               stop_sound_playing()
        elif is_backside_salute_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, head_top,):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        elif is_backside_hands_up_pose(left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        elif is_backside_jesus_christ_pose(left_shoulder, right_shoulder,left_wrist, right_wrist):
                label, color = "Friendly Pose Detected", (0, 255, 0)
                stop_sound_playing()
        else:
            label, color = "Alert! Shooting Triggered!", (0, 0, 255)
            start_sound()
        
            # Handle friendly poses
   
        # Track the time when a pose is detected
        if "Pose Detected" in label or label == "Alert! Shooting Triggered!":
            if friendly_pose_time is None:
                friendly_pose_time = time.time()  # Start tracking time

    # Display the detected label
            color = (0, 255, 0) if "Pose Detected" in label else (0, 0, 255)
            cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # After 2 seconds, display move direction message
            if time.time() - friendly_pose_time >= 2:
                move_direction = "Choose option Move Left or Right"
                cv2.putText(frame, move_direction, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        else:
          friendly_pose_time = None  # Reset when no pose is detected


   
        
    frame_count += 1
    

    cv2.imshow('Video', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        stop_sound = True
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
pygame.quit()
